In [4]:
import numpy as np
import nibabel as nib 
import os 
import pandas as pd
from utils.utils import * 
from utils.flips import get_prefered_xy
import math

In [5]:
"""
Export for the full brain fits is different: we do not make a model, we pick on sample ROI and export all the fitted values on that one target 
"""

rois_long = {'V1v': 1, 'V1d': 2, 'V2v': 3, 'V2d': 4, 'V3v': 5, 'V3d': 6, 'hV4': 7, 'VO-1': 8,
 'VO-2': 9, 'PHC-1': 10, 'PHC-2': 11, 'LO-1': 12, 'LO-2': 13, 'TO-1': 14, 'TO-2': 15
}
sior_long = {v: k for k,v in rois_long.items()}
to_reduce = {'V1v': 'V1', 'V1d': 'V1', 'V2v': 'V2', 'V2d': 'V2', 'V3v': 'V3', 'V3d': 'V3'}
sior = {v: k for k,v in rois.items()}

In [6]:
cos_sin = get_prefered_xy(subj_list, sessions, fetch_conds=False) # save that and open it matlab 

(907,)


In [27]:

models_subs = {}

columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"]

## Change this line to select which sampling space you want for the full brain model. We chose TO-1 since it was the most common sampling space
sampling_space = 'TO-1'
mode = "train"
rotated = True


for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    participant_df = pd.DataFrame(columns=columns)
    for k in range(5):
        fits_file = os.path.join(fits_dir, 'fits_inversed', sub,  f'fits_{sub}_{mode}_{sampling_space}_full_{k}.npy')

        chunk_df = pd.DataFrame(np.load(fits_file, allow_pickle=True, mmap_mode='r'), columns=columns)
        
        participant_df = pd.concat([participant_df, chunk_df], ignore_index=True)
        
    models_subs[sub] = participant_df
  #  models_subs[sub][columns[:-2]] = models_subs[sub][columns[:-2]].astype(np.float32)
        # this should be okay &
    models_subs[sub]['x0_prefered'] = models_subs[sub]['x0'] * cos_sin[i, 0]-models_subs[sub]['y0'] * cos_sin[i, 1] # the mds have different dimension -> so problem 
    models_subs[sub]['y0_prefered'] = models_subs[sub]['x0'] * cos_sin[i, 1]+models_subs[sub]['y0'] * cos_sin[i, 0]

    sub_m_study = models_subs[sub]
hier_index = pd.MultiIndex.from_arrays(
    [
        np.tile(list(rois.keys()), len(models_subs)),
        np.repeat(list(models_subs.keys()), len(rois.keys())),
    ],
    names =["roi", "sub"]
)
models_subs["subj01"]["test_var_explained"]

# check distribution of values in VO-1s for common conditions 

models_subs

{'subj01':               x0        y0      sigma      slope  intercept  \
 0       0.125106 -0.575907  17.208112   0.000012  -0.006038   
 1      -1.049999 -1.050000   0.519903   0.219634  -0.043687   
 2       1.049998  1.050000   5.880621   2.379431  -2.332390   
 3       1.050000  1.050000  20.425045  28.186732 -28.138634   
 4      -1.050000 -1.050000   0.447156   0.187168  -0.029636   
 ...          ...       ...        ...        ...        ...   
 453599  1.023147  1.049315   0.010740   0.000001  -0.002546   
 453600  0.051728  0.546409   0.012147   0.939146  -0.021818   
 453601 -1.049351  0.488163   3.475119   0.450898  -0.433589   
 453602 -1.049908 -0.013990   1.331855   0.113324  -0.070451   
 453603 -1.049742  0.734001   9.144259   3.387640  -3.372838   
 
         test_var_explained  var_explained   mds_ecc   mds_ang  x0_prefered  \
 0                -0.000328  -9.536743e-07  1.057427  2.927683     0.125106   
 1                -0.003223   4.195929e-03  1.484924 -2.356195

In [32]:
hemis = ['lh', 'rh']
fields = ['test_var_explained']
fields.append('x0_prefered')
fields.append('y0_prefered')
if rotated:
    rotation = "rotated"
if not rotated: 
    rotation = "notrotated"


for sub in subj_list:
        m = models_subs[sub]
        print(f'------ EXPORTING MODEL FOR {sub}')
        for f in fields:
            print(f'\t{f}')
            s = -1
            for hemi in hemis:
                print(f'\t\t{hemi}')
                
                
                maskdata_long_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.{sub}.nans_full.testrois.mgz') # should be the right mask
                maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
                
                print(maskdata_long.shape)
        
                maskdata_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.short.reduced.{sub}.testrois.npy')
                maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)

                data_out_file = os.path.join(results_dir, sub, f'{hemi}.{f}.full{sampling_space}.mgz')
                label_out_file = os.path.join(label_dir,'freesurfer', sub, 'label', f'{hemi}.{f}.full{sampling_space}.mgz') # annoying but necessary for matlab output
                if os.path.exists(data_out_file):
                    pass
                data_out = np.zeros(maskdata_long.shape)
             
                wrongs = []
                for i in range(maskdata_long.shape[0]):
                    voxel = int(maskdata_long[i])
                    if voxel ==-100:
                      
                        continue
                    s += 1
                    if f == 'mds_ang':
                        data_out[i] = math.degrees(m.iloc[s][f]) + 180
                    elif f == 'intercept':
                        data_out[i] = m.iloc[s][f] + abs(m[f].min()) + 1e-08
                    elif f in ['x0', 'y0', 'x0_prefered', 'y0_prefered']:
                        data_out[i] = m.iloc[s][f] + 1.1
                    elif f in ['var_explained', 'test_var_explained']:  # This is a bit of a hack, but it mask values too close to 0, which is nice for illustration
                        if m.iloc[s][f]  > -0.003 and m.iloc[s][f]  < 0.0003:
                            data_out[i] =  - 10
                        else:
                            data_out[i] = m.iloc[s][f]  + 1 # Add 1 for the variance explain, so we can still mask the non relevant value and plot the "negative" variance
                    else:
                        data_out[i] = m.iloc[s][f]  
                img = nib.Nifti1Image(np.expand_dims(data_out, axis=(1, 2)), np.eye(4))

                nib.loadsave.save(img, data_out_file)
                nib.loadsave.save(img, label_out_file)
                if f == 'mds_ang':
                    for ang in [45, 90, 135, 180, 225, 270, 305]:
                        data_out_file = os.path.join(results_dir, sub, f'{hemi}.{f}.{str(ang)}.full{sampling_space}.mgz')
                        label_out_file = os.path.join(label_dir, 'freesurfer', sub, 'label', f'{hemi}.{f}.{str(ang)}.full{sampling_space}.mgz')
                        data_out_ang = np.copy(data_out)
                        data_out_ang[data_out != 0] += ang
                        data_out_ang = data_out_ang % 360
                        img = nib.Nifti1Image(np.expand_dims(data_out_ang, axis = (1, 2)), np.eye(4))
                        nib.loadsave.save(img, data_out_file)
                        nib.loadsave.save(img, label_out_file)


print('end')
                        
            

------ EXPORTING MODEL FOR subj01
	test_var_explained
		lh
(227021,)
		rh
(226601,)
	x0_prefered
		lh
(227021,)
		rh
(226601,)
	y0_prefered
		lh
(227021,)
		rh
(226601,)
------ EXPORTING MODEL FOR subj02
	test_var_explained
		lh
(239633,)
		rh
(239309,)
	x0_prefered
		lh
(239633,)
		rh
(239309,)
	y0_prefered
		lh
(239633,)
		rh
(239309,)
------ EXPORTING MODEL FOR subj03
	test_var_explained
		lh
(240830,)
		rh
(243023,)
	x0_prefered
		lh
(240830,)
		rh
(243023,)
	y0_prefered
		lh
(240830,)
		rh
(243023,)
------ EXPORTING MODEL FOR subj04
	test_var_explained
		lh
(228495,)
		rh
(227262,)
	x0_prefered
		lh
(228495,)
		rh
(227262,)
	y0_prefered
		lh
(228495,)
		rh
(227262,)
------ EXPORTING MODEL FOR subj05
	test_var_explained
		lh
(197594,)
		rh
(198908,)
	x0_prefered
		lh
(197594,)
		rh
(198908,)
	y0_prefered
		lh
(197594,)
		rh
(198908,)
------ EXPORTING MODEL FOR subj06
	test_var_explained
		lh
(253634,)
		rh
(259406,)
	x0_prefered
		lh
(253634,)
		rh
(259406,)
	y0_prefered
		lh
(2536